In [1]:
# import comet_ml at the top of your file
from comet_ml import Experiment
experiment = Experiment(
            project_name="music-genre-multiclass-classification",
            workspace="wodenwang820118",
        )
import comet_ml
import logging

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/4025d21d069d452a879ec95dacaf1b05



In [ ]:
logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger("comet_ml")

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow import keras

%matplotlib inline
sns.set_style('whitegrid')

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [ ]:
# tensorflow 2.7 
import tensorflow as tf 
from tensorflow.keras.layers import Input,Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model, Sequential

In [ ]:
# scale the numeric data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
df_train_features = pd.read_csv('data/train_features.csv')
df_train_labels = pd.read_csv('data/train_labels.csv')

df_valid_features = pd.read_csv('data/valid_features.csv')
df_valid_labels = pd.read_csv('data/valid_labels.csv')

df_test_features = pd.read_csv('data/test_features.csv')
df_test_labels = pd.read_csv('data/test_labels.csv')

In [ ]:
num_train_data = df_train_features.iloc[:,9:]
num_valid_data = df_valid_features.iloc[:,9:]
num_test_data = df_test_features.iloc[:,9:]

In [ ]:
num_train = num_train_data.astype('float64')
num_valid = num_valid_data.astype('float64')
num_test = num_test_data.astype('float64')

In [ ]:
num_train = scaler.fit_transform(num_train_data)
num_valid = scaler.fit_transform(num_valid_data)
num_test = scaler.fit_transform(num_test_data)

In [ ]:
num_train_y = pd.get_dummies(df_train_labels['genre'])
num_valid_y = pd.get_dummies(df_valid_labels['genre'])

In [ ]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(df_train_labels['genre']),y=df_train_labels['genre'])
class_weight = {i :class_weight[i] for i in range(8)}

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=5)

In [ ]:
class AudioGenreClassifier:
    def __init__(self,num_train,num_train_y,num_valid,num_valid_y,class_weight,early_stop,experiment):
        self.num_train = num_train
        self.num_train_y = num_train_y
        self.num_valid = num_valid
        self.num_valid_y = num_valid_y
        self.class_weight = class_weight
        self.early_stop = early_stop
        self.experiment = experiment
    
    def build_model(self):
        # Build the model
        model = Sequential()
        model.add(
            Dense(
                self.experiment.get_parameter("first_layer_units"),
                activation='elu',
                input_shape=(num_train.shape[1],)
            )
        )
        model.add(Dropout(self.experiment.get_parameter("first_layer_dropout_rate")))
        model.add(BatchNormalization())

        model.add(
            Dense(
                self.experiment.get_parameter("second_layer_units"),
                activation='elu'))
        model.add(Dropout(self.experiment.get_parameter("second_layer_dropout_rate")))
        model.add(BatchNormalization())
        
        model.add(Dense(8,activation='softmax'))
        model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        return model
    
    def train_model(self):
        # Train the model
        model = self.build_model()
        model.fit(
            self.num_train,
            self.num_train_y,
            batch_size=self.experiment.get_parameter("batch_size"),
            epochs=self.experiment.get_parameter("epochs"),
            validation_data=(self.num_valid,self.num_valid_y),
            class_weight=self.class_weight,
            callbacks=[self.early_stop]
        )
        return model
    
    def evaluate_model(self):
        # Evaluate the model
        model = self.train_model()
        score = model.evaluate(self.num_valid,self.num_valid_y)
        LOGGER.info(f"{ score }")
    
    def grid_search(self, config_dict):
        opt = comet_ml.Optimizer(config_dict)
        for self.experiment in opt.get_experiments(project_name="music-genre-multiclass-classification"):
            self.experiment.log_parameters("epochs", 10)

            self.build_model()
            self.train_model()
            self.evaluate_model()
            self.experiment.end()

In [ ]:
audio_model = AudioGenreClassifier(num_train,num_train_y,num_valid,num_valid_y,class_weight,early_stop,experiment)
# mu is the mean number of units, sigma is the standard deviation
audio_model.grid_search({
    "algorithm": "bayes",
    "name": "Optimize Music Classification Network",
    "spec": {"maxCombo": 10, "objective": "minimize", "metric": "loss"},
    "parameters": {
        "first_layer_units": {"type": "discrete", "values": [128,256,300,400,450,500,550,600,700,800,900,1000]},
        "first_layer_dropout_rate": {"type": "discrete", "values": [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]},
        "second_layer_units": {"type": "discrete", "values": [128,256,300,400,450,500,550]},
        "second_layer_dropout_rate": {"type": "discrete", "values": [0,0.1,0.2,0.3,0.4,0.5,0.6]},
        "batch_size": {"type": "discrete", "values": [16,32,64]},
        "epochs": {"type": "discrete", "values": [20,30,40]},
    },
    "trials": 1,
})

COMET INFO: COMET_OPTIMIZER_ID=879bfda029bc40219d0bf59cb1535707
COMET INFO: Using optimizer config: {'algorithm': 'bayes', 'configSpaceSize': 63, 'endTime': None, 'id': '879bfda029bc40219d0bf59cb1535707', 'lastUpdateTime': None, 'maxCombo': 10, 'name': 'Optimize Music Classification Network', 'parameters': {'batch_size': {'type': 'discrete', 'values': [16, 32, 64]}, 'epochs': {'type': 'discrete', 'values': [20, 30, 40]}, 'first_layer_units': {'type': 'discrete', 'values': [128, 256, 300, 400, 450, 500, 550]}}, 'predictor': None, 'spec': {'gridSize': 10, 'maxCombo': 10, 'metric': 'loss', 'minSampleSize': 100, 'objective': 'minimize', 'retryAssignLimit': 0, 'retryLimit': 1000}, 'startTime': 27540902528, 'state': {'mode': None, 'seed': None, 'sequence': [], 'sequence_i': 0, 'sequence_pid': None, 'sequence_retry': 0, 'sequence_retry_count': 0}, 'status': 'running', 'suggestion_count': 0, 'trials': 1, 'version': '2.0.1'}
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experimen

Epoch 1/40


COMET INFO: ignoring tensorflow summary log of metrics because of keras; set `comet_ml.loggers.tensorboard_logger.LOG_METRICS = True` to override


120/120 [==============================] - 4s 13ms/step - loss: 1.7355 - accuracy: 0.3509 - val_loss: 1.4223 - val_accuracy: 0.4778
Epoch 2/40
120/120 [==============================] - 1s 9ms/step - loss: 1.4821 - accuracy: 0.4415 - val_loss: 1.4576 - val_accuracy: 0.4733
Epoch 3/40
120/120 [==============================] - 1s 10ms/step - loss: 1.4203 - accuracy: 0.4693 - val_loss: 1.3723 - val_accuracy: 0.5089
Epoch 4/40
120/120 [==============================] - 1s 11ms/step - loss: 1.3609 - accuracy: 0.4884 - val_loss: 1.4155 - val_accuracy: 0.4778
Epoch 5/40
120/120 [==============================] - 1s 10ms/step - loss: 1.3189 - accuracy: 0.4975 - val_loss: 1.3666 - val_accuracy: 0.5089
Epoch 6/40
120/120 [==============================] - 1s 9ms/step - loss: 1.3078 - accuracy: 0.5020 - val_loss: 1.3702 - val_accuracy: 0.4933
Epoch 7/40
120/120 [==============================] - 2s 13ms/step - loss: 1.2701 - accuracy: 0.5218 - val_loss: 1.3230 - val_accuracy: 0.5289
Epoch 8/40
1

COMET INFO: [1.3665591478347778, 0.5088889002799988]
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/0196687d45964f7dadcd28ba7123a0b4
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [24]                : (0.35087263584136963, 0.5408960580825806)
COMET INFO:     batch_accuracy [288]         : (0.0625, 0.609375)
COMET INFO:     batch_loss [288]             : (0.8443261384963989, 2.985323429107666)
COMET INFO:     epoch_duration [24]          : (0.9539999999997235, 3.969000000000051)
COMET INFO:     loss [24]                    : (1.2168821096420288, 1.7510747909545898)
COMET INFO:     val_accuracy [24]            : (0.46444445848464966, 0.551111102104187)
COMET INFO:     val_loss [24]                : (1.2986239194869995, 1.

Epoch 1/20
480/480 [==============================] - 6s 11ms/step - loss: 1.7780 - accuracy: 0.3442 - val_loss: 1.4530 - val_accuracy: 0.4511
Epoch 2/20
480/480 [==============================] - 5s 10ms/step - loss: 1.5424 - accuracy: 0.4164 - val_loss: 1.4388 - val_accuracy: 0.4644
Epoch 3/20
480/480 [==============================] - 5s 11ms/step - loss: 1.4808 - accuracy: 0.4311 - val_loss: 1.4323 - val_accuracy: 0.4689
Epoch 4/20
480/480 [==============================] - 6s 12ms/step - loss: 1.4180 - accuracy: 0.4624 - val_loss: 1.4084 - val_accuracy: 0.5044
Epoch 5/20
480/480 [==============================] - 6s 12ms/step - loss: 1.4283 - accuracy: 0.4561 - val_loss: 1.4350 - val_accuracy: 0.4756
Epoch 6/20
480/480 [==============================] - 5s 11ms/step - loss: 1.4034 - accuracy: 0.4638 - val_loss: 1.3837 - val_accuracy: 0.5133
Epoch 7/20
480/480 [==============================] - 5s 11ms/step - loss: 1.3807 - accuracy: 0.4819 - val_loss: 1.3413 - val_accuracy: 0.5244

In [ ]:
model = Sequential()
model.add(
    Dense(
        400,
        activation='elu',
        input_shape=(num_train.shape[1],)
    )
)
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(32,activation='elu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(8,activation='softmax'))
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.fit(
    num_train,
    num_train_y,
    batch_size=64,
    epochs=40,
    validation_data=(num_valid,num_valid_y),
    class_weight=class_weight,
    callbacks=[early_stop],
)

Epoch 1/40
120/120 [==============================] - 1s 7ms/step - loss: 1.7323 - accuracy: 0.3442 - val_loss: 1.4725 - val_accuracy: 0.4444
Epoch 2/40
120/120 [==============================] - 1s 4ms/step - loss: 1.4953 - accuracy: 0.4315 - val_loss: 1.4363 - val_accuracy: 0.4578
Epoch 3/40
120/120 [==============================] - 0s 4ms/step - loss: 1.3996 - accuracy: 0.4700 - val_loss: 1.4465 - val_accuracy: 0.4467
Epoch 4/40
120/120 [==============================] - 0s 3ms/step - loss: 1.3769 - accuracy: 0.4859 - val_loss: 1.3990 - val_accuracy: 0.4867
Epoch 5/40
120/120 [==============================] - 0s 4ms/step - loss: 1.3462 - accuracy: 0.5005 - val_loss: 1.3450 - val_accuracy: 0.5089
Epoch 6/40
120/120 [==============================] - 0s 3ms/step - loss: 1.3144 - accuracy: 0.5046 - val_loss: 1.3227 - val_accuracy: 0.5400
Epoch 7/40
120/120 [==============================] - 0s 3ms/step - loss: 1.3045 - accuracy: 0.5138 - val_loss: 1.3691 - val_accuracy: 0.5200
Epoch 

In [ ]:
from sklearn.metrics import classification_report
validation = model.predict(num_valid)
validation = validation.argmax(axis=1)
classes_mapping = {
    0: 'classic pop and rock',
    1: 'dance and electronica',
    2: 'folk',
    3: 'jazz and blues',
    4: 'metal',
    5: 'pop',
    6: 'punk',
    7: 'soul and reggae',
}
predict_label_array = np.vectorize(classes_mapping.get)(validation)
correct_labels = df_valid_labels['genre'].values
print(classification_report(correct_labels,predict_label_array))

                       precision    recall  f1-score   support

 classic pop and rock       0.34      0.45      0.39        55
dance and electronica       0.41      0.40      0.40        45
                 folk       0.42      0.53      0.47        64
       jazz and blues       0.54      0.50      0.52        44
                metal       0.92      0.68      0.78        66
                  pop       0.56      0.42      0.48        74
                 punk       0.49      0.70      0.58        44
      soul and reggae       0.70      0.52      0.59        58

             accuracy                           0.52       450
            macro avg       0.55      0.53      0.53       450
         weighted avg       0.56      0.52      0.53       450



In [ ]:
from sklearn.metrics import classification_report
validation = model.predict(num_test)
validation = validation.argmax(axis=1)
classes_mapping = {
    0: 'classic pop and rock',
    1: 'dance and electronica',
    2: 'folk',
    3: 'jazz and blues',
    4: 'metal',
    5: 'pop',
    6: 'punk',
    7: 'soul and reggae',
}
predict_label_array = np.vectorize(classes_mapping.get)(validation)
correct_labels = df_test_labels['genre'].values
print(classification_report(correct_labels,predict_label_array))

                       precision    recall  f1-score   support

 classic pop and rock       0.24      0.31      0.27        64
dance and electronica       0.55      0.36      0.43        64
                 folk       0.58      0.58      0.58        73
       jazz and blues       0.13      0.14      0.14        36
                metal       0.49      0.43      0.46        42
                  pop       0.33      0.48      0.39        44
                 punk       0.27      0.33      0.29        40
      soul and reggae       0.50      0.32      0.39        65

             accuracy                           0.38       428
            macro avg       0.39      0.37      0.37       428
         weighted avg       0.41      0.38      0.39       428

